<a href="https://colab.research.google.com/github/rafabelokurows/similar-countries/blob/main/World_Bank_similarities_between_countries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
pd.set_option('display.float_format', '{:.3f}'.format)

In [ ]:
wb = pd.read_csv('worldbank 2911.csv')

In [ ]:
wb.head()

,country,iso2c,iso3c,year,status,lastupdated,NY.GDP.PCAP.KD,SE.XPD.TOTL.GD.ZS,GB.XPD.RSDV.GD.ZS,NY.GDP.TOTL.RT.ZS,...,SI.POV.LMIC,SE.SEC.NENR,VC.IHR.PSRC.P5,SG.GEN.PARL.ZS,region,capital,longitude,latitude,income,lending
0,Afghanistan,AF,AFG,1999,NaN,2022-09-16,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,South Asia,Kabul,69.176,34.523,Low income,IDA
1,Afghanistan,AF,AFG,2018,NaN,2022-09-16,546.743,3.200,NaN,0.891,...,NaN,50.141,6.656,NaN,South Asia,Kabul,69.176,34.523,Low income,IDA
2,Afghanistan,AF,AFG,2019,NaN,2022-09-16,555.139,3.214,NaN,0.693,...,NaN,NaN,NaN,27.869,South Asia,Kabul,69.176,34.523,Low income,IDA
3,Afghanistan,AF,AFG,2018,NaN,2020-09-21,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,South Asia,Kabul,69.176,34.523,Low income,IDA
4,Afghanistan,AF,AFG,1998,NaN,2022-09-16,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,South Asia,Kabul,69.176,34.523,Low income,IDA


## Removendo alguns duplicados
".first()" equivalente ao slice(1) do R

In [ ]:
wb2 = wb.groupby(["country","year"]).first()

## Contando quantos registros tem por ano e a completeness (%)
25 indicadores - caso buscar mais indicadores, mudar este número

In [ ]:
byYear = wb2.groupby(['year']).count()
byYear['complete'] = byYear.iloc[:,4:-7].sum(axis=1)
byYear['completeness'] = (byYear.complete/25)/byYear.iso2c

## Filtrando só os anos que tem completeness > 60%

In [ ]:
byYear.sort_values('completeness',ascending=False).query('completeness > 0.6').head()

,iso2c,iso3c,status,lastupdated,NY.GDP.PCAP.KD,SE.XPD.TOTL.GD.ZS,GB.XPD.RSDV.GD.ZS,NY.GDP.TOTL.RT.ZS,GC.XPN.TOTL.GD.ZS,EN.ATM.CO2E.KD.GD,...,VC.IHR.PSRC.P5,SG.GEN.PARL.ZS,region,capital,longitude,latitude,income,lending,complete,completeness
year,,,,,,,,,,,,,,,,,,,,,
2017,267,263,0,268,254,218,123,256,162,234,...,122,238,259,211,211,211,259,259,4779,0.716
2012,267,263,0,268,252,178,113,255,155,233,...,196,236,259,211,211,211,259,259,4497,0.674
2015,267,263,0,268,258,209,129,257,164,236,...,174,237,259,211,211,211,259,259,4456,0.668
2018,267,263,0,268,254,209,117,256,155,234,...,111,239,259,211,211,211,259,259,4454,0.667
2016,267,263,0,268,254,209,116,256,162,234,...,126,240,259,211,211,211,259,259,4368,0.654


## Selecionando dados para a primeira comparação
* Preenchendo NAs
* Selecionando dados de 2017 (apenas alguns países)
* Selecionando apenas colunas de interesse
* Dropando NAs

In [ ]:
wb3=wb2.fillna(method='ffill').query('year == 2017').iloc[:,np.r_[4:8,10:18,20:24]].dropna().query("country in ('Cuba','Belize','Angola','Egypt','Australia','India','South Africa','Kenya','Uganda','Portugal','Brazil','Argentina','United States','Colombia','China','Japan','Canada','Spain','Italy','France','Mexico','Chile','Uruguay','Paraguay')")

## Normalizando dados

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(wb3), columns=wb3.columns)
df_scaled.head()

,NY.GDP.PCAP.KD,SE.XPD.TOTL.GD.ZS,GB.XPD.RSDV.GD.ZS,NY.GDP.TOTL.RT.ZS,GC.DOD.TOTL.GD.ZS,SH.XPD.CHEX.GD.ZS,SH.XPD.GHED.GD.ZS,NE.CON.PRVT.ZS,NV.IND.TOTL.ZS,MS.MIL.XPND.GD.ZS,GC.TAX.TOTL.GD.ZS,SM.POP.NETM,SP.DYN.LE00.IN,SP.DYN.IMRT.IN,SH.STA.BASS.ZS,SP.POP.TOTL
0,0.035,0.020,0.000,1.000,0.417,0.000,0.066,0.473,1.000,0.720,0.000,0.362,0.000,1.000,0.378,0.021
1,0.222,0.302,0.152,0.066,0.417,0.546,0.606,0.716,0.293,0.146,0.101,0.361,0.674,0.141,0.943,0.031
2,0.991,0.272,0.536,0.306,0.253,0.462,0.582,0.452,0.352,0.543,0.757,0.464,0.933,0.025,1.000,0.017
3,0.067,0.491,1.000,0.056,0.384,0.214,0.339,0.854,0.000,0.285,1.000,0.359,0.590,0.192,0.849,0.000
4,0.133,0.384,0.315,0.172,0.415,0.477,0.337,0.659,0.169,0.340,0.258,0.372,0.636,0.231,0.847,0.149


## Calculando a similaridade com o método do cosseno

In [ ]:
similar=cosine_similarity(df_scaled) #calculando a similaridade 
similar.shape #ver formato do NP array
similar[np.diag_indices_from(similar)] = np.NAN
df=pd.DataFrame(data=similar[:,:],index=wb3.index,columns=wb3.index.get_level_values(0))
df['mostSimilar'] = df.idxmax(axis=1)
df['similarity'] = df.max(axis=1)
df2 = df[['mostSimilar','similarity']]

<ipython-input-67-feb12348a9b6>:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df['similarity'] = df.max(axis=1)


In [204]:
df2.droplevel(1).style.background_gradient(cmap = "BuGn").set_caption("Most similar countries to each other based on key indicators")

country,mostSimilar,similarity
country,,
Angola,Uganda,0.870279
Argentina,Brazil,0.960603
Australia,France,0.959391
Belize,Portugal,0.879493
Brazil,Argentina,0.960603
Canada,Spain,0.967922
Chile,Uruguay,0.964135
China,India,0.811634
Colombia,Chile,0.945049


## Países mais semelhantes ao Brasil

In [203]:
brazilData = df.droplevel(1).query("country == 'Brazil'").iloc[:,:-2]
brazilData.reset_index(level=0,drop=True).melt(ignore_index=False).reset_index().query("country != 'Brazil'").iloc[:,1:].rename(columns={"value": "similarity"}).style.background_gradient(cmap = "BuGn")
brazilData.reset_index(level=0,drop=True).melt(ignore_index=False).reset_index().query("country != 'Brazil'").iloc[:,1:].rename(columns={"value": "similarity"}).sort_values('similarity',ascending=False).style.background_gradient(cmap = "BuGn").set_caption("Most similar countries to Brazil based on key indicators").format({'similarity': "{:.3}"})
#pd.wide_to_long(brazilData, ["A", "B"], i=brazilData.index, j="year")
# abc = pd.DataFrame({
#     'date' : ['05/03', '06/03', '07/03', '08/03'],
#     'AA' : [1, 4, 7, 5],
#     'BB' : [2, 5, 8, 7],
#     'CC' : [3, 6, 9, 1]
# }).set_index('date').reset_index()
# abc
#countries = list(['Cuba','Belize','Angola','Australia','India','South Africa','Kenya','Uganda','Portugal','Brazil','Argentina','United States','Colombia','China','Japan','Canada','Spain','Italy','France','Mexico','Chile','Uruguay','Paraguay'])
#pd.melt(brazilData, id_vars='country', value_vars=['Cuba','Belize','Angola','Australia','India','South Africa','Kenya','Uganda','Portugal','Brazil','Argentina','United States','Colombia','China','Japan','Canada','Spain','Italy','France','Mexico','Chile','Uruguay','Paraguay'])


#.query('').pivot(index='country',columns='year',values='NY.GDP.PCAP.KD')
#wbWide.columns = wbWide.columns.map(lambda x: f'GDP_{x+1}')

,country,similarity
1,Argentina,0.961
22,Uruguay,0.946
19,Spain,0.937
6,Chile,0.925
8,Colombia,0.915
17,Portugal,0.908
5,Canada,0.904
12,Italy,0.898
10,France,0.89
15,Mexico,0.883


In [ ]:
#Preenchendo NAs
# wb2 = wb.fillna(method="backfill").fillna(method='ffill')
#Nulos por coluna
# wb2.isnull().sum()
# cols=['country','year','NY.GDP.PCAP.KD']
# wb3.reset_index(drop=True,inplace=True)
#Pivot para transformar o PandasFrame em Wide
# wbWide = wb2.loc[:,cols].pivot(index='country',columns='year',values='NY.GDP.PCAP.KD')
# wbWide.columns = wbWide.columns.map(lambda x: f'GDP_{x+1}')
# wb3.pivot(index='country',columns='year',values='NY.GDP.PCAP.KD')